In [3]:
from PIL import Image

import json
import os
import shutil
import unicodedata
import uuid
import cv2
import mediapipe as mp
from tqdm.auto import tqdm

/home/ant/miniconda3/envs/psl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def create_new_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)           # Removes all the subdirectories!
        print('Directory existed and was cleaned')
        os.makedirs(path)

In [5]:
folder_path_raw = '../dataset/images_raw'
folder_path = '../dataset/images'

In [6]:
def convert_to_jpg(file_path, path_to_save):
    img = Image.open(file_path)
    if img.format == 'PNG': # Conversion of png to jpg
        file_name = os.path.splitext(path_to_save)[0]
        img = img.convert("RGB")
        img.save(file_name + '.jpg', 'JPEG')
    else:
        file_name = os.path.splitext(path_to_save)[0]
        img.save(file_name + '.jpg', 'JPEG')

create_new_folder(folder_path)

for file_name in tqdm(os.listdir(folder_path_raw)):
    if file_name.lower().endswith('.png') or  file_name.lower().endswith('.jpg'):
        file_path = os.path.join(folder_path_raw, file_name)
        path_to_save = os.path.join(folder_path, file_name)
        convert_to_jpg(file_path, path_to_save)

Directory existed and was cleaned


100%|██████████| 3741/3741 [02:10<00:00, 28.60it/s]


In [7]:
with os.scandir(folder_path) as entries: # Used scandir instead of listdir to detect Polish characters (e.g ą,ę)
    print(len(list(entries)))

3702


In [8]:
def extract_labels(folder_path):
    labels = {}
    # /home/ant/projects/psl/dataset/images/A35.jpg
    skipped = 0
    created = 0
    labels_folder = '../dataset/labels/'
    images_path = '../dataset/images_renamed'
    
    create_new_folder(labels_folder)
    create_new_folder(images_path)

    # Initialize MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True)
    mp_drawing = mp.solutions.drawing_utils

    # Searching for files in a folder
    with os.scandir(folder_path) as entries: # Used scandir instead of listdir to detect Polish characters (e.g ą,ę)
        for entry in tqdm(entries):
            if entry.is_file() and entry.name.lower().endswith('.jpg'): # I want to exclude files that are not jpg
                filename = entry.name
                #print(filename)
                label = ''.join([c for c in filename.split('.')[0] if not c.isdigit()])
                
                # Skip file names starting with 'img'
                if filename.lower().startswith('img'):
                    # print(f'skipping starting with img {filename}')
                    skipped += 1
                    continue
                
                # Normalize the filename to ensure proper handling of diacritical marks
                normalized_filename = unicodedata.normalize('NFC', filename)

                # Check if filename contains 'sz', 'cz', 'ch' or 'rz'
                if 'sz' in filename:
                    labels[filename] = 'sz'
                elif 'cz' in filename:
                    labels[filename] = 'cz'
                elif 'ch' in filename:
                    labels[filename] = 'ch'
                elif 'rz' in filename:
                    labels[filename] = 'rz'
                else:    
                    # Retrieving the first letter of the file name    
                    labels[normalized_filename] = label
                
                # Prepare the label dictionary for the current image
                image_labels = {'label': label}
                
                new_file_name = str(uuid.uuid4()).replace('-', '')
                shutil.copyfile(os.path.join(folder_path, filename), os.path.join(images_path, new_file_name + '.jpg'))

                # Process image using MediaPipe: loading image
                image = cv2.imread(os.path.join(folder_path, filename))  # Loading the image from disk
                # Convert the image to RGB format (MediaPipe requires RGB input)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # Processing the image using MediaPipe Hands
                results = hands.process(image_rgb)

                # Get landmarks and add them to the label dictionary
                if results.multi_hand_landmarks:  # Checking if hand landmarks exist in the image processing results
                    for hand_landmarks in results.multi_hand_landmarks:
                        landmarks = hand_landmarks.landmark  # Retrieving the hand landmarks
                        
                        # Creating a dictionary containing the coordinates of the hand landmarks
                        landmark_dict = {f'hand_landmark_{i}': {'x': landmark.x, 'y': landmark.y, 'z': landmark.z} for i, landmark in enumerate(landmarks)}
                        image_labels['hand_landmarks'] = landmark_dict  # Adding the hand landmarks to the label dictionary for the image
                
               # Saving the results to a JSON file
                with open(os.path.join(labels_folder, f'{new_file_name}.json'), 'w', encoding='utf-8') as json_file:
                    json.dump(image_labels, json_file,  indent=4, ensure_ascii=False)
                created += 1
            else:
                skipped += 1  
    print(created, skipped)

# Calling the function
extract_labels(folder_path)

I0000 00:00:1704367923.163823   85283 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1704367923.180162   85453 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.129.03), renderer: NVIDIA GeForce GTX 970/PCIe/SSE2


Directory existed and was cleaned
Directory existed and was cleaned


0it [00:00, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
3702it [02:19, 26.52it/s]

3626 76


In [39]:
def count_files_with_landmarks(labels_folder):
    labeled_with_landmarks_count = 0
    labeled_without_landmarks_count = 0
    data_rows = []
    # Iterate through files in the labels folder
    for filename in tqdm(os.listdir(labels_folder)):
        if filename.endswith('.json'):  # Check if the file is a JSON file
            with open(os.path.join(labels_folder, filename), 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                # Check if the JSON file contains the 'landmarks' section
                if 'hand_landmarks' in data:
                    labeled_with_landmarks_count += 1

                    # Inspect the structure of 'hand_landmarks'
                    landmarks_data = data['hand_landmarks']
                    # print(landmarks_data)  # Print or inspect the structure
                        
                    # Inicjalizacja wiersza dla każdego punktu charakterystycznego
                    row = []
                    
                    # Dodawanie współrzędnych x, y, z każdego punktu do wiersza
                    for landmark_key in landmarks_data:
                        landmark = landmarks_data[landmark_key]
                        row.extend([landmark['x'], landmark['y'], landmark['z']])
                    
                    # Dodanie etykiety (label) na końcu wiersza
                    row.append(data['label'])  # Użyj label jako etykiety (label)
                    
                    # Dodanie wiersza do listy danych
                    data_rows.append(row)

                    # print(data_rows[0])  # Wyświetlenie pierwszego wiersza dla przykładu

                else:
                    labeled_without_landmarks_count += 1
    print(data_rows[0])
    print(len(data_rows[0]))
    return labeled_with_landmarks_count, labeled_without_landmarks_count

In [40]:
labels_path= '../dataset/labels'
number_of_labeled_files_with_landmarks, number_of_labeled_files_without_landmarks = count_files_with_landmarks(labels_path)
print(f'Number of JSON files with landmarks information: {number_of_labeled_files_with_landmarks}')
print(f'Number of JSON files without landmarks information: {number_of_labeled_files_without_landmarks}')

  0%|          | 0/3626 [00:00<?, ?it/s]

100%|██████████| 3626/3626 [00:00<00:00, 16977.86it/s]

[0.4261322021484375, 0.832544207572937, -2.4167698597921117e-07, 0.513171911239624, 0.747363805770874, -0.02008882537484169, 0.5800948143005371, 0.629132091999054, -0.03888978809118271, 0.6436914801597595, 0.5610277056694031, -0.06331326812505722, 0.7082765102386475, 0.5351800918579102, -0.09024006873369217, 0.48088496923446655, 0.4791983962059021, -0.0426514558494091, 0.494485080242157, 0.36818814277648926, -0.07652100175619125, 0.5026534795761108, 0.2925238013267517, -0.09785345941781998, 0.5090571045875549, 0.2255845069885254, -0.11323678493499756, 0.4086472988128662, 0.4774588942527771, -0.054321013391017914, 0.47094079852104187, 0.5469447374343872, -0.11051315069198608, 0.49115514755249023, 0.6556470394134521, -0.12455194443464279, 0.4916194975376129, 0.7292878031730652, -0.12253744155168533, 0.3522123098373413, 0.5256809592247009, -0.06698207557201385, 0.42579904198646545, 0.6243754625320435, -0.11562752723693848, 0.4508973956108093, 0.704166054725647, -0.11300017684698105, 0.458